In [1]:
#!/usr/bin/env python3.6
# -*- coding: utf-8 -*-

In [2]:
# Imports
import sys
import math
import numpy as np
from matplotlib import animation
from IPython.display import HTML
from matplotlib import pyplot as plt
plt.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg'
import mpl_toolkits.mplot3d.axes3d as p3

In [3]:
np.random.seed(20)
np.set_printoptions(threshold=sys.maxsize)

In [4]:
%matplotlib inline

# Data

In [5]:
# Read data
path = '../../../dataset4_wbc/live_extraction/aliengo/step_0.065/accelerations_1m.csv'
motion = np.genfromtxt(path, delimiter=',', dtype=np.float64)
contacts = np.genfromtxt(path, delimiter=',', usecols=(29, 30, 31, 32), dtype=bool)
print("Motion data has a length of ", len(motion))

Motion data has a length of  13718


In [6]:
print(motion.shape)

(13718, 33)


# Dataset Preparation

In [7]:
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [8]:
def quaternion_rotation_matrix(Q):
    """
    Covert a quaternion into a full three-dimensional rotation matrix.
 
    Input
    :param Q: A 4 element array representing the quaternion (q0,q1,q2,q3) 
 
    Output
    :return: A 3x3 element matrix representing the full 3D rotation matrix. 
             This rotation matrix converts a point in the local reference 
             frame to a point in the global reference frame.
    """
    # Extract the values from Q
    q0 = Q[3]
    q1 = Q[0]
    q2 = Q[1]
    q3 = Q[2]
     
    # First row of the rotation matrix
    r00 = 2 * (q0 * q0 + q1 * q1) - 1
    r01 = 2 * (q1 * q2 - q0 * q3)
    r02 = 2 * (q1 * q3 + q0 * q2)
     
    # Second row of the rotation matrix
    r10 = 2 * (q1 * q2 + q0 * q3)
    r11 = 2 * (q0 * q0 + q2 * q2) - 1
    r12 = 2 * (q2 * q3 - q0 * q1)
     
    # Third row of the rotation matrix
    r20 = 2 * (q1 * q3 - q0 * q2)
    r21 = 2 * (q2 * q3 + q0 * q1)
    r22 = 2 * (q0 * q0 + q3 * q3) - 1
     
    # 3x3 rotation matrix
    rot_matrix = np.array([[r00, r01, r02],
                           [r10, r11, r12],
                           [r20, r21, r22]])
                            
    return rot_matrix

In [15]:
def absolute_footstep_accelerations_dataset(dataset, footsteps, debug=False):
    # Footsteps sizes
    inputs = []
    labels = []
    
    for idx, footstep in enumerate(footsteps):
        if idx > 0:
            # Compute time difference between
            # curhent and next footsteps
            time_difference = abs(dataset[footsteps[idx], 0] - dataset[footsteps[idx-1], 0])

            if time_difference < 0.5:            
                fl_moving = contacts[footsteps[idx], 0]
                fr_moving = contacts[footsteps[idx], 1]
                rl_moving = contacts[footsteps[idx], 2]
                rr_moving = contacts[footsteps[idx], 3]
                
                assert fr_moving == rl_moving
                assert rr_moving == fl_moving
                assert fr_moving != fl_moving

                # Compute rotation matrices from odometry orientation
                R_prev = quaternion_rotation_matrix(dataset[footsteps[idx-1], 19:23])
                R_curr = quaternion_rotation_matrix(dataset[footsteps[idx], 19:23])

                # Odom base vectors
                prev_base = np.array([[dataset[footsteps[idx-1], 16], dataset[footsteps[idx-1], 17], 0]])
                curr_base = np.array([[dataset[footsteps[idx], 16], dataset[footsteps[idx], 17], 0]])

                # Rotate odometry estimation
                rotated_prev_base = R_prev.T @ prev_base.T
                rotated_curr_base = R_prev.T @ curr_base.T
                base_displacement = rotated_curr_base - rotated_prev_base
                
                inputs.append(dataset[footsteps[idx-1], 1:4].tolist() +
                              dataset[footsteps[idx-1], 23:29].tolist() +
                              #dataset[footsteps[idx-1], 4:16].tolist() + 
                              (dataset[footsteps[idx-1], 4:16].reshape(4,3) - dataset[footsteps[idx-1], 16:19].reshape(1,3)).reshape(12,).tolist() +
                              [fl_moving, fr_moving, rl_moving, rr_moving] +
                              dataset[footsteps[idx], 1:4].tolist())
                    
                if fl_moving and rr_moving:
                    # FL vectors
                    prev_fl = np.array([[dataset[footsteps[idx-1], 4], dataset[footsteps[idx-1], 5], 0]])
                    curr_fl = np.array([[dataset[footsteps[idx], 4], dataset[footsteps[idx], 5], 0]])

                    # RR vectors
                    prev_rr = np.array([[dataset[footsteps[idx-1], 13], dataset[footsteps[idx-1], 14], 0]])
                    curr_rr = np.array([[dataset[footsteps[idx], 13], dataset[footsteps[idx], 14], 0]])

                    # Rotate FL vector
                    rotated_prev_fl = R_prev.T @ prev_fl.T
                    rotated_curr_fl = R_prev.T @ curr_fl.T
                    fl_displacement = rotated_curr_fl - rotated_prev_fl

                    # Rotate RR vector
                    rotated_prev_rr = R_prev.T @ prev_rr.T
                    rotated_curr_rr = R_prev.T @ curr_rr.T
                    rr_displacement = rotated_curr_rr - rotated_prev_rr
   
                    labels.append([fl_displacement[0,0], fl_displacement[1,0], rr_displacement[0,0], rr_displacement[1,0]])
                    #labels.append([rotated_curr_fl[0,0]-rotated_curr_base[0,0], rotated_curr_fl[1,0]-rotated_curr_base[1,0], 
                    #               rotated_curr_rr[0,0]-rotated_curr_base[0,0], rotated_curr_rr[1,0]-rotated_curr_base[1,0]])
                else:
                    # FR vectors
                    prev_fr = np.array([[dataset[footsteps[idx-1], 7], dataset[footsteps[idx-1], 8], 0]])
                    curr_fr = np.array([[dataset[footsteps[idx], 7], dataset[footsteps[idx], 8], 0]])

                    # RL vectors
                    prev_rl = np.array([[dataset[footsteps[idx-1], 10], dataset[footsteps[idx-1], 11], 0]])
                    curr_rl = np.array([[dataset[footsteps[idx], 10], dataset[footsteps[idx], 11], 0]])

                    # Rotate FR vector
                    rotated_prev_fr = R_prev.T @ prev_fr.T
                    rotated_curr_fr = R_prev.T @ curr_fr.T
                    fr_displacement = rotated_curr_fr - rotated_prev_fr

                    # Rotate RL vector
                    rotated_prev_rl = R_prev.T @ prev_rl.T
                    rotated_curr_rl = R_prev.T @ curr_rl.T
                    rl_displacement = rotated_curr_rl - rotated_prev_rl

                    labels.append([fr_displacement[0,0], fr_displacement[1,0], rl_displacement[0,0], rl_displacement[1,0]])
                    #labels.append([rotated_curr_fr[0,0]-rotated_curr_base[0,0], rotated_curr_fr[1,0]-rotated_curr_base[1,0], 
                    #               rotated_curr_rl[0,0]-rotated_curr_base[0,0], rotated_curr_rl[1,0]-rotated_curr_base[1,0]])
                    
    # Convert to numpy array
    inputs_numpy = np.array(inputs)
    labels_numpy = np.array(labels)
    
    return inputs_numpy, labels_numpy

# Stack datasets
X_footsteps_accelerations_absolute, Y_footsteps_accelerations_absolute = absolute_footstep_accelerations_dataset(motion, np.array([x for x in range(len(motion))]))

# Log
print(X_footsteps_accelerations_absolute.shape)
print(Y_footsteps_accelerations_absolute.shape)

(13679, 28)
(13679, 4)


In [16]:
%store X_footsteps_accelerations_absolute
%store Y_footsteps_accelerations_absolute

Stored 'X_footsteps_accelerations_absolute' (ndarray)
Stored 'Y_footsteps_accelerations_absolute' (ndarray)


In [11]:
Y_footsteps_accelerations_absolute[:10, :]

array([[ 0.23179327,  0.14004969, -0.24871657, -0.13906732],
       [ 0.23066972, -0.15635525, -0.24881498,  0.15832435],
       [ 0.23104885,  0.15769172, -0.24856249, -0.15704255],
       [ 0.23021587, -0.16368494, -0.24929886,  0.1654614 ],
       [ 0.23074399,  0.16525878, -0.24879558, -0.16299948],
       [ 0.24020646, -0.16584804, -0.23911132,  0.17027016],
       [ 0.23853637,  0.16813658, -0.24077563, -0.16782352],
       [ 0.24362152, -0.16779473, -0.23559274,  0.17154128],
       [ 0.24136077,  0.16968122, -0.23783159, -0.16965005],
       [ 0.25302584, -0.1674824 , -0.22677623,  0.1730809 ]])

In [12]:
max(Y_footsteps_accelerations_absolute[:, 0])

0.2772540854181216

In [13]:
min(Y_footsteps_accelerations_absolute[:, 0])

-0.06112655272343659

In [14]:
x = np.where(X_footsteps_accelerations_absolute[:, 1] > 0)[0]

In [15]:
Y_footsteps_accelerations_absolute[x, 0]

array([-1.69641300e-03, -3.57593359e-03, -3.26582544e-03, -4.68302562e-03,
       -4.23522213e-03, -9.08225183e-03, -6.26964447e-03, -9.33025433e-03,
       -6.73968290e-03, -5.68612284e-03, -5.54124220e-03, -6.54992139e-03,
       -5.79044999e-03, -1.05565815e-02, -7.29492920e-03, -9.44653455e-03,
       -7.01913088e-03, -9.01386206e-03, -1.01239924e-02, -1.06780397e-02,
       -8.37735875e-03, -9.37205627e-03, -5.46871440e-03, -6.50132637e-03,
       -5.73271909e-03, -8.29113506e-03, -8.90067202e-03, -9.99689384e-03,
       -7.99872110e-03, -8.77985348e-03, -7.59590517e-03, -1.24165007e-02,
       -9.25183610e-03, -1.04528172e-02, -8.03736342e-03, -6.33276227e-03,
       -6.28832361e-03, -6.86581520e-03, -6.16722496e-03, -1.08535749e-02,
       -7.58824450e-03, -9.46832133e-03, -7.40133547e-03, -5.12981627e-03,
       -5.45067605e-03, -6.57600883e-03, -5.76529266e-03, -1.12012519e-02,
       -7.86087996e-03, -8.61459177e-03, -7.55681726e-03, -5.54937749e-03,
       -5.37052077e-03, -